# QPU and QPU Topology

In a typical experiment, the quantum elements that we would like to control are on a quantum processing unit (QPU) with a given topology, which describes how the quantum elements are connected to each other. Understanding the properties of the QPU is important in a variety of situations, such as applying multi-qubit gates or compensating for crosstalk.

In LabOne Q, we use the [dsl.QPU](https://docs.zhinst.com/labone_q_user_manual/core/reference/dsl/quantum.html#laboneq.dsl.quantum.qpu.QPU) class to organize this information, which takes the given quantum elements and quantum operations as input arguments. The default QPU topology is then constructed on initialization from these quantum elements and is accessible via the `topology` attribute. Additional connections in the topology can be made at a later stage.

In this tutorial, we will go through the basic properties of the `QPU` class, including: how to initialize a QPU, how to modify its topology, and how this may be applied in the context of a real experiment.

## Imports

In [ ]:
import attrs

from laboneq.core.utilities.dsl_dataclass_decorator import classformatter
from laboneq.dsl.quantum import (
    QPU,
    Transmon,
    QuantumOperations,
    QuantumParameters,
    QuantumElement,
)
from laboneq.simple import *

## Define the quantum elements and operations

Following from the previous tutorials, we will demonstrate the functionality of the QPU class using `Transmon` qubits. We start by defining an example qubit template together with a set of example qubit operations.

In [ ]:
def qubit_template(i):
    return Transmon(
        uid=f"q{i}",
        signals={
            "drive": f"q{i}/drive",
            "measure": f"q{i}/measure",
            "acquire": f"q{i}/acquire",
        },
        parameters={"resonance_frequency_ge": i},
    )


class TransmonOperations(QuantumOperations):
    QUBIT_TYPES = Transmon

In addition to qubits, we may also have other quantum elements on the QPU. For example, we define a custom `Coupler` quantum element below, together with its associated `CouplerOperations`.

In [ ]:
@classformatter
@attrs.define(kw_only=True)
class CouplerParameters(QuantumParameters):
    amplitude: float = 0.5
    length: float = 100e-9
    pulse: dict = attrs.field(factory=lambda: {"function": "gaussian_square"})


class Coupler(QuantumElement):
    PARAMETERS_TYPE = CouplerParameters
    REQUIRED_SIGNALS = ("flux",)


class CouplerOperations(QuantumOperations):
    QUBIT_TYPES = Coupler


c0 = Coupler(
    uid="c0",
    signals={"flux": "c0/flux"},
)

## Define the QPU

A `QPU` object can be defined from a single quantum element, a sequence of quantum elements, or a dictionary of quantum element groups, together with a subclass, or list, of `QuantumOperations`. For example, we can define a QPU simply using a list of quantum elements, as shown below.

<div class="alert alert-block alert-info">
<b>Note:</b>

If a list of quantum operations is given, these are combined into a single class. For further details, please see the tutorial on [combining quantum operations](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/04_quantum_processing_unit/tutorials/00_quantum_operations.html#combining-quantum-operations).
</div>

In [ ]:
quantum_element_list = [qubit_template(i) for i in range(2)] + [c0]
qpu = QPU(
    quantum_elements=quantum_element_list,
    quantum_operations=[TransmonOperations, CouplerOperations],
)
qpu

In this case, we can see that we have three quantum elements on our QPU with UIDs: `q0`, `q1`, `c0`. This corresponds to two transmon qubits and one tunable coupler in the experiment. We can access these quantum elements directly from the QPU by UID, slice, or subclass.

In [ ]:
qpu["q0"]  # returns a single quantum element by UID
qpu[["q0", "q1"]]  # returns a list of quantum elements by UID
qpu[:2]  # returns the first two quantum elements by slice
qpu[Transmon]  # returns the quantum elements of a given type

If there are multiple kinds of quantum elements present in the QPU, then it may be useful to categorise them into groups. The advantage of this is that we can then conveniently retrieve these custom groups as attributes of `qpu.groups`. The behaviour of the QPU is otherwise unaffected.

In [ ]:
quantum_element_dict = {
    "qubits": [qubit_template(i) for i in range(2)],
    "couplers": [c0],
}
qpu = QPU(
    quantum_elements=quantum_element_dict,
    quantum_operations=[TransmonOperations, CouplerOperations],
)
qpu.groups.qubits
qpu.groups.couplers

The instructions that our QPU supports are the `TransmonOperations` that we defined above. There are no connections between our quantum elements by default and so there are currently no edges in the `topology` attribute. 

## Define the QPU topology

By default, all of the quantum elements on the QPU are initialized as disconnected nodes in the QPU topology. We can check this by plotting the initial QPU topology graph with `disconnected=True`. Note that the `quantum_elements` argument for `QPU` is a complete list of all the quantum elements present on the QPU and therefore, nodes cannot be added or removed from the topology after the QPU has been defined.

In [ ]:
qpu.topology.plot(disconnected=True)

### Nodes

The information about the nodes can be looked up using the `nodes` and `node_keys` iterators. The `nodes` iterator generates the quantum elements at the nodes, and the `node_keys` iterator generates the UIDs of the quantum_elements at the nodes.

In [ ]:
for node in qpu.topology.nodes():
    print(node)

In [ ]:
for node_key in qpu.topology.node_keys():
    print(node_key)

It is also possible to retrieve the information on a specific node in the graph using the `get_node` method.

In [ ]:
qpu.topology.get_node("q0")

<div class="alert alert-block alert-info">
<b>Note:</b>

The node retrieval methods in the `QPUTopology` class are provided for completeness. Accessing the nodes in `QPUTopology` is discouraged, in favour of equivalent methods in the `QPU` class. 
    
For example, we recommend using `qpu["q0"]` instead of `qpu.topology.get_node("q0")`. 
</div>

### Edges

Since the nodes represent the complete set of quantum elements on the QPU, and therefore cannot be changed after the QPU is defined, modifications to the QPU topology come in the form of adding and removing edges. An edge is a directed connection between two nodes. Optionally, an edge may also have its own set of parameters and/or its own associated quantum element.

<div class="alert alert-block alert-info">
<b>Note:</b>
The quantum element associated to an edge may only come from the pool of quantum elements that are present on the QPU.
</div>

Since there may be multiple edges between two nodes on the QPU, we provide each edge with a user-defined string called a `tag`. In this way, an edge may be accessed via the tuple `(tag, source_node, target_node)`, where `tag` is a user-defined string, `source_node` is the UID of the source node, and `target_node` is the UID of the target node.

Here, we will look at a few examples to demonstrate how this works. We start by adding a single edge between nodes 0 and 1. The edge appears on the graph as an arrow going from the source to the target node. The edge tag is labeled on the arrow. Analogously, edges may be removed using the `remove_edge` method.

In [ ]:
qpu.topology.add_edge("empty", "q0", "q1")
qpu.topology.plot()

In this fashion, we can continue to add edges to the graph until the topology of the QPU is accurately described. For example, we can add an additional edge from node 0 to node 1, and we can add an edge in the opposite direction, from node 1 to node 0. Each edge may also have a set of edge parameters and its own quantum element. In the example below, we add the coupler `c0` to the edge going from `q0` to `q1`. For clarity, the edge quantum element UID is printed next to the edge tag.

In [ ]:
qpu.topology.add_edge("coupler", "q0", "q1", quantum_element="c0")
qpu.topology.add_edge("empty", "q1", "q0")
qpu.topology.plot()

Similar to the nodes, information about the edges may be looked up using the `edges` and `edge_keys` iterators. The `edges` iterator generates the edges in the graph, which are `TopologyEdge` objects, and the `edge_keys` iterator generates the keys of the edges, which are the `(tag, source_node, target_node)` tuples.

In [ ]:
for edges in qpu.topology.edges():
    print(edges)

In [ ]:
for edge_key in qpu.topology.edge_keys():
    print(edge_key)

It is also possible to retrieve the information on a particular edge directly from the QPU topology.

In [ ]:
qpu.topology["coupler", "q0", "q1"]

Alternatively, we can retrieve information on multiple edges by replacing one or more of the edge key elements with null slices. For example, we can list all of the outgoing edges from "q0".

In [ ]:
qpu.topology[:, "q0", :]

To improve the plot readability, we can fix the positions of the quantum elements, set an equal aspect ratio, and omit the edge tags.

In [ ]:
qpu.topology.plot(
    fixed_pos={"q0": (0, 0), "q1": (1, 0)}, equal_aspect=True, show_tags=False
)

We can also check and filter the list of neighboring nodes using the `neighbors` method. Using this, we can check for example, whether all qubits are connected before performing a quantum operation.

In [ ]:
qpu.topology.neighbors("q0")

#### Edge parameters

We have seen above that the edges are directional `(tag, source_node, target_node)`. This feature together with the ability to attach parameters to an edge is useful for defining directional two-qubit gates, such as `iSWAP_q0_q1` and `iSWAP_q1_q0`. Let's see how we can do this using edge parameters.

In the example below, we pass a custom `IswapParameters` class to the `add_edge` method.

<div class="alert alert-block alert-info">
<b>Note:</b>
Mutable default values are discouraged in python. To avoid this, we initialize dictionary attributes using `attrs` factories.
</div>

For further instructions on how to define your own subclass of `QuantumParameters`, please see the [Quantum Elements tutorial](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/04_quantum_processing_unit/tutorials/01_quantum_elements.html#define-the-parameters).

In [ ]:
def default_control_pulse():
    return {
        "function": "gaussian_square",
        "amplitude": 0.9,
        "length": 100e-9,
    }


def default_coupler_pulse():
    return {
        "function": "const",
        "amplitude": 0.9,
        "length": 100e-9,
    }


@classformatter
@attrs.define(kw_only=True)
class IswapParameters(QuantumParameters):
    control_pulse: dict = attrs.field(factory=default_control_pulse)
    coupler_pulse: dict = attrs.field(factory=default_coupler_pulse)

Next, we create two new edges that are both called `iswap` and have the coupler `c0` as a quantum element. One edge is directed from `q0` to `q1` and the other from `q1` to `q0`. We add the `IswapParameters` to both edges:

In [ ]:
qpu.topology.add_edge(
    "iswap", "q0", "q1", quantum_element=c0, parameters=IswapParameters()
)
qpu.topology.add_edge(
    "iswap", "q1", "q0", quantum_element=c0, parameters=IswapParameters()
)
qpu.topology.plot()

We can now set different values for the parameters of the two edges, corresponding to the different implementations of those two gates:

In [ ]:
qpu.topology["iswap", "q0", "q1"].parameters.control_pulse["amplitude"] = 0.25
qpu.topology["iswap", "q0", "q1"].parameters.coupler_pulse["amplitude"] = 0.25

qpu.topology["iswap", "q1", "q0"].parameters.control_pulse["amplitude"] = 0.5
qpu.topology["iswap", "q1", "q0"].parameters.coupler_pulse["amplitude"] = 0.5

In [ ]:
qpu.topology["iswap", "q0", "q1"]

In [ ]:
qpu.topology["iswap", "q1", "q0"]

The parameters for the quantum elements and/or topology edges may also be modified at a later stage using the `QPU.update` method. For example, if we would like to set the `iswap` edge amplitudes in the example above both to `0.5`, but the amplitude of the coupler to `1`, we may write:

In [ ]:
new_parameters = {
    ("iswap", "q0", "q1"): {
        "control_pulse": {"amplitude": 0.5},
        "coupler_pulse": {"amplitude": 0.5},
    },
    "c0": {"amplitude": 1},
}
qpu.update(new_parameters)

In [ ]:
qpu.topology["iswap", "q0", "q1"]

In [ ]:
qpu["c0"]

## Saving/loading the QPU

Finally, we can view the summary information for our newly-defined QPU by printing the `QPU` object.

In [ ]:
qpu

Here, we can see that we have three edge tags in our topology graph: `empty`, `coupler`, and `iswap`. Since `empty` and `iswap` appear twice and `coupler` appears once, our topology graph has five edges in total.

Once we are finished, the QPU object may be saved and loaded just like other quantum objects in LabOne Q, using the `save`/`load` methods from `laboneq.serializers`.

For further information on designing your own experiment in LabOne Q, please see the [LabOne Q Applications Library](https://docs.zhinst.com/labone_q_user_manual/applications_library/index.html).